In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
#Combine user data and user comments so password_id can be used to identify botnets

user_data_df = pd.read_csv("/kaggle/input/disqus-social-bot-networks/anonymized_disqus_user_data.csv")
user_comments_df = pd.read_csv("/kaggle/input/disqus-social-bot-networks/botnet_comments.csv")
user_meta_data_df = pd.read_csv("/kaggle/input/disqus-social-bot-networks/user_meta_data.csv")
user_comments_df = user_comments_df.merge(user_data_df, how='left')
user_comments_df = user_comments_df.merge(user_meta_data_df, how='left')

In [ ]:
#password_423651 previously identified as an influence campaign
arab_spring_comments_df = user_comments_df.loc[user_comments_df['password_id'] == 'password_423651']
print(len(arab_spring_comments_df))

In [ ]:
comments = arab_spring_comments_df["message"]
cleaned_messages = arab_spring_comments_df.apply(lambda row : cleanhtml(row['message']), axis = 1)

comments = cleaned_messages.tolist() 
comments = ' '.join(comments).lower()

wordcloud = WordCloud(stopwords = STOPWORDS,
                      collocations=True).generate(comments)

plt.imshow(wordcloud, interpolation='bilInear')
plt.axis('off')
plt.show()

In [ ]:
#List all profile urls
arab_spring_comments_df = arab_spring_comments_df.sort_values(by="numPosts", ascending=False)
no_dupes = arab_spring_comments_df.drop_duplicates(subset='username', keep="last")
profileurls = no_dupes.apply (lambda row: "https://disqus.com/by/" + str(row["username"]), axis=1)

print(profileurls.to_string())